# 입문자를 위한, 파이썬/R 데이터 분석  

## Part(2) - Day15  : 

]

## Today's mission

- FDR를 통한 여러 종목 수익률 비교
- 네이버 금융 개별종목 수집
- plotly로 일별 시세 시각화

]

## 네이버 금융 개별종목 수집
* FinanceDataReader를 통해 수집했던 데이터를 네이버 증권 웹 페이지를 통해 직접 수집합니다.


### Keyword

* html 파일 읽어오기
    * pd.read_html(url, encoding="cp949")

* 결측 데이터 제거하기(axis 0:행, 1:열)
    * table[0].dropna()

* 데이터 프레임 합치기
    * pd.concat([df1, df2, df3])

* 중복데이터 제거
    * df.drop_duplicates()

* 과학적 기수법
    * 1.210000e+02 => 121

* 날짜 column의 첫 row값 확인
    * date = df.iloc[0]["날짜"]

* 파일로 저장하기 
    * df.to_csv(file_name, index=False)

* 파일 읽어오기
    * pd.read_csv(file_name)

## 수집할 페이지 보기

* 네이버 금융 국내증시 : https://finance.naver.com/sise/
* 2020년 주요 상장종목
    * 하이브 : https://finance.naver.com/item/main.nhn?code=352820
    * 카카오게임즈 : https://finance.naver.com/item/main.nhn?code=293490
    * SK바이오팜 : https://finance.naver.com/item/main.nhn?code=326030

## 라이브러리 로드

In [1]:
# 라이브러리 로드
import pandas as pd

## 수집할 URL 정하기

In [2]:
# 종목번호와 상장사 이름을 item_code와 item_name으로 설정
# item_code = "352820"
# item_name = "빅히트"

item_code = "326030"
item_name = "SK바이오팜"
page_no = 1

# 종목 URL 만들기
url = f"https://finance.naver.com/item/sise_day.nhn?code={item_code}&page={page_no}"
url

'https://finance.naver.com/item/sise_day.nhn?code=326030&page=1'

## requests를 통한 HTTP 요청
* [Requests: HTTP for Humans™ — Requests documentation](https://requests.readthedocs.io/en/master/)
* [Quickstart — Requests documentation # custom-headers](https://requests.readthedocs.io/en/latest/user/quickstart/#custom-headers)

In [3]:
import requests

headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36'}

response = requests.get(url, headers=headers)

In [4]:
response

<Response [200]>

In [5]:
response.text

'\n<html lang="ko">\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=euc-kr">\n<title>네이버페이 증권</title>\n\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20240808165745/css/newstock.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20240808165745/css/common.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20240808165745/css/finance_header.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20240808165745/css/layout.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20240808165745/css/main.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20240808165745/css/newstock2.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20240808165745/css/newstock3.css">\n<link rel="st

## BeautifulSoup 을 통한 table 태그 찾기

* [Beautiful Soup Documentation — Beautiful Soup 4.9.0 documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

In [6]:
from bs4 import BeautifulSoup as bs

html = bs(response.text, "lxml")
table = html.select("table")
len(table)

2

In [7]:
type(table)

bs4.element.ResultSet

## pandas 코드 한 줄로 데이터 수집하기

In [8]:
# read_html을 이용하여 url의 page내의 값을 DataFrame으로 받아옵니다.
# cp949는 한글 인코딩을 위해 사용합니다. 기본 인코딩 설정은 utf-8 이며, 
# 네이버의 일별 시세는 cp949 인코딩으로 불러올 수 있습니다.
# 데이터를 로드 했을 때 한글 인코딩이 깨진다면 대부분 cp949 로 불러올 수 있습니다.
table = pd.read_html(str(table))

C:\Users\salix\AppData\Local\Temp\ipykernel_15824\3976437143.py:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))


In [9]:
str(table)

'[            날짜       종가        전일비       시가       고가       저가       거래량\n0          NaN      NaN        NaN      NaN      NaN      NaN       NaN\n1   2024.08.14  97000.0        보합0  97400.0  98900.0  96300.0   93944.0\n2   2024.08.13  97000.0  하락  1,800  98000.0  99800.0  95300.0  393782.0\n3   2024.08.12  98800.0  상승  6,500  94200.0  99200.0  93000.0  707165.0\n4   2024.08.09  92300.0  상승  2,500  91000.0  93400.0  89300.0  342745.0\n5   2024.08.08  89800.0  상승  4,300  84500.0  92900.0  82800.0  698388.0\n6          NaN      NaN        NaN      NaN      NaN      NaN       NaN\n7          NaN      NaN        NaN      NaN      NaN      NaN       NaN\n8          NaN      NaN        NaN      NaN      NaN      NaN       NaN\n9   2024.08.07  85500.0  상승  6,400  77600.0  85700.0  77500.0  180833.0\n10  2024.08.06  79100.0  상승  1,700  79300.0  82200.0  78200.0  163093.0\n11  2024.08.05  77400.0  하락  8,700  85000.0  85300.0  72600.0  214229.0\n12  2024.08.02  86100.0  하락  1,000  86100.0  8790

In [10]:
# table[0]와 table[1]을 확인하여 보면 table[0]에 필요한 데이터들이 있습니다.
table[0]

,날짜,종가,전일비,시가,고가,저가,거래량
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024.08.14,97000.0,보합0,97400.0,98900.0,96300.0,93944.0
2,2024.08.13,97000.0,"하락 1,800",98000.0,99800.0,95300.0,393782.0
3,2024.08.12,98800.0,"상승 6,500",94200.0,99200.0,93000.0,707165.0
4,2024.08.09,92300.0,"상승 2,500",91000.0,93400.0,89300.0,342745.0
5,2024.08.08,89800.0,"상승 4,300",84500.0,92900.0,82800.0,698388.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2024.08.07,85500.0,"상승 6,400",77600.0,85700.0,77500.0,180833.0


In [11]:
table[1]

,0,1,2,3,4,5,6,7,8,9,10,11
0,1,2,3,4,5,6,7,8,9,10,다음,맨뒤


In [12]:
# dropna를 통해 결측치가 들어있는 row를 제거합니다.
temp = table[0].dropna()
temp

,날짜,종가,전일비,시가,고가,저가,거래량
1,2024.08.14,97000.0,보합0,97400.0,98900.0,96300.0,93944.0
2,2024.08.13,97000.0,"하락 1,800",98000.0,99800.0,95300.0,393782.0
3,2024.08.12,98800.0,"상승 6,500",94200.0,99200.0,93000.0,707165.0
4,2024.08.09,92300.0,"상승 2,500",91000.0,93400.0,89300.0,342745.0
5,2024.08.08,89800.0,"상승 4,300",84500.0,92900.0,82800.0,698388.0
9,2024.08.07,85500.0,"상승 6,400",77600.0,85700.0,77500.0,180833.0
10,2024.08.06,79100.0,"상승 1,700",79300.0,82200.0,78200.0,163093.0
11,2024.08.05,77400.0,"하락 8,700",85000.0,85300.0,72600.0,214229.0
12,2024.08.02,86100.0,"하락 1,000",86100.0,87900.0,85100.0,119535.0
13,2024.08.01,87100.0,"상승 1,400",85800.0,88400.0,85300.0,101239.0


## 페이지별 데이터 수집 함수 만들기

In [13]:
# 종목 번호를 이용해 page에 따라 데이터를 읽어오는 함수
# """ 는 이 두개 사이의 행들은 주석 처리되며, 함수의 docstring 으로 사용됩니다.
def get_day_list(item_code, page_no):
    """
    일자별 시세를 페이지별로 수집
    """ 
    url = f"https://finance.naver.com/item/sise_day.nhn?code={item_code}&page={page_no}"
    headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36'}
    response = requests.get(url, headers=headers)
    html = bs(response.text, "lxml")
    table = html.select("table")
    table = pd.read_html(str(table))
    temp = table[0].dropna()
    return temp

In [14]:
# 함수가 잘 만들어졌는지 확인하기
get_day_list(item_code, 3)

C:\Users\salix\AppData\Local\Temp\ipykernel_15824\4100071483.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))


,날짜,종가,전일비,시가,고가,저가,거래량
1,2024.07.17,85100.0,"상승 3,300",83300.0,86400.0,82900.0,263057.0
2,2024.07.16,81800.0,보합0,82200.0,82700.0,80500.0,71493.0
3,2024.07.15,81800.0,"하락 1,200",83800.0,83800.0,81700.0,73241.0
4,2024.07.12,83000.0,"상승 2,700",80500.0,83800.0,80100.0,145779.0
5,2024.07.11,80300.0,상승 300,80900.0,81000.0,79100.0,87512.0
9,2024.07.10,80000.0,하락 700,79900.0,81000.0,79400.0,55437.0
10,2024.07.09,80700.0,상승 800,80300.0,81200.0,79700.0,98503.0
11,2024.07.08,79900.0,상승 900,80100.0,80900.0,78700.0,116454.0
12,2024.07.05,79000.0,"상승 1,900",77200.0,79800.0,76700.0,118693.0
13,2024.07.04,77100.0,하락 400,77700.0,78500.0,76300.0,68848.0


## 반복문을 통한 전체 일자 데이터 수집하기
* (주의) 기간이 긴 데이터를 수집할때는 서버에 부담을 주지 않기 위해 time.sleep()값을 주세요.

In [15]:
temp.shape[0]

10

In [16]:
import time
# web page 시작번호
page_no = 1
# 데이터를 저장할 빈 변수 선언
item_list = []

while True:
    print(page_no)
    temp = get_day_list(item_code, page_no)
    item_list.append(temp)
    
    page_no = page_no + 1
    time.sleep(1)
    
    if temp.shape[0] < 10:    # 맨마지막 데이터는 10행미만
    #if page_no > 10:
        break;

C:\Users\salix\AppData\Local\Temp\ipykernel_15824\4100071483.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))


1
2


C:\Users\salix\AppData\Local\Temp\ipykernel_15824\4100071483.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))


3


C:\Users\salix\AppData\Local\Temp\ipykernel_15824\4100071483.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))


...

C:\Users\salix\AppData\Local\Temp\ipykernel_15824\4100071483.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))


101


C:\Users\salix\AppData\Local\Temp\ipykernel_15824\4100071483.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))


102


C:\Users\salix\AppData\Local\Temp\ipykernel_15824\4100071483.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))


In [17]:
len(item_list)

102

In [18]:
type(item_list)

list

In [19]:
item_list[-1]
#item_list

,날짜,종가,전일비,시가,고가,저가,거래량
1,2020.07.10,205500.0,보합0,200500.0,213500.0,198500.0,1825361.0
2,2020.07.09,205500.0,"하락 11,500",213500.0,220000.0,205500.0,1897646.0
3,2020.07.08,217000.0,상승 500,214500.0,225000.0,198000.0,5990009.0
4,2020.07.07,216500.0,"상승 2,000",219000.0,269500.0,210000.0,10105187.0
5,2020.07.06,214500.0,"상한가 49,500",214500.0,214500.0,193500.0,7126211.0
9,2020.07.03,165000.0,"상한가 38,000",165000.0,165000.0,165000.0,711921.0
10,2020.07.02,127000.0,"상한가 29,000",98000.0,127000.0,98000.0,698642.0


## 수집한 데이터 하나의 데이터프레임으로 합치기

<img src="https://pandas.pydata.org/docs/_images/merging_concat_basic.png">

* [Merge, join, concatenate and compare documentation](https://pandas.pydata.org/docs/user_guide/merging.html#merge-join-concatenate-and-compare)

In [20]:
# DataFrame들이 list의 형태로 저장된 것을 concat을 이용하여 합치면서 하나의 DataFrame으로 만듭니다.
df = pd.concat(item_list)

<img src="https://pandas.pydata.org/docs/_images/08_concat_row.svg">

In [21]:
# head와 tail로 데이터의 일부를 가져와서 봅니다.
df.head()

,날짜,종가,전일비,시가,고가,저가,거래량
1,2024.08.14,97000.0,보합0,97400.0,98900.0,96300.0,93944.0
2,2024.08.13,97000.0,"하락 1,800",98000.0,99800.0,95300.0,393782.0
3,2024.08.12,98800.0,"상승 6,500",94200.0,99200.0,93000.0,707165.0
4,2024.08.09,92300.0,"상승 2,500",91000.0,93400.0,89300.0,342745.0
5,2024.08.08,89800.0,"상승 4,300",84500.0,92900.0,82800.0,698388.0


In [22]:
df.tail()

,날짜,종가,전일비,시가,고가,저가,거래량
3,2020.07.08,217000.0,상승 500,214500.0,225000.0,198000.0,5990009.0
4,2020.07.07,216500.0,"상승 2,000",219000.0,269500.0,210000.0,10105187.0
5,2020.07.06,214500.0,"상한가 49,500",214500.0,214500.0,193500.0,7126211.0
9,2020.07.03,165000.0,"상한가 38,000",165000.0,165000.0,165000.0,711921.0
10,2020.07.02,127000.0,"상한가 29,000",98000.0,127000.0,98000.0,698642.0


In [23]:
pd.set_option('display.max_row', 8000)
df

,날짜,종가,전일비,시가,고가,저가,거래량
1,2024.08.14,97000.0,보합0,97400.0,98900.0,96300.0,93944.0
2,2024.08.13,97000.0,"하락 1,800",98000.0,99800.0,95300.0,393782.0
3,2024.08.12,98800.0,"상승 6,500",94200.0,99200.0,93000.0,707165.0
4,2024.08.09,92300.0,"상승 2,500",91000.0,93400.0,89300.0,342745.0
5,2024.08.08,89800.0,"상승 4,300",84500.0,92900.0,82800.0,698388.0
9,2024.08.07,85500.0,"상승 6,400",77600.0,85700.0,77500.0,180833.0
10,2024.08.06,79100.0,"상승 1,700",79300.0,82200.0,78200.0,163093.0
11,2024.08.05,77400.0,"하락 8,700",85000.0,85300.0,72600.0,214229.0
12,2024.08.02,86100.0,"하락 1,000",86100.0,87900.0,85100.0,119535.0
13,2024.08.01,87100.0,"상승 1,400",85800.0,88400.0,85300.0,101239.0


## 데이터프레임에 종목코드와 종목명을 추가하기
* 파생변수 만들기

In [24]:
# '종목코드'와 '종목명' column을 추가하면서 각각 item_code와 item_name 값을 입력합니다.
df["종목코드"] = item_code
df["종목명"] = item_name
df

,날짜,종가,전일비,시가,고가,저가,거래량,종목코드,종목명
1,2024.08.14,97000.0,보합0,97400.0,98900.0,96300.0,93944.0,326030,SK바이오팜
2,2024.08.13,97000.0,"하락 1,800",98000.0,99800.0,95300.0,393782.0,326030,SK바이오팜
3,2024.08.12,98800.0,"상승 6,500",94200.0,99200.0,93000.0,707165.0,326030,SK바이오팜
4,2024.08.09,92300.0,"상승 2,500",91000.0,93400.0,89300.0,342745.0,326030,SK바이오팜
5,2024.08.08,89800.0,"상승 4,300",84500.0,92900.0,82800.0,698388.0,326030,SK바이오팜
9,2024.08.07,85500.0,"상승 6,400",77600.0,85700.0,77500.0,180833.0,326030,SK바이오팜
10,2024.08.06,79100.0,"상승 1,700",79300.0,82200.0,78200.0,163093.0,326030,SK바이오팜
11,2024.08.05,77400.0,"하락 8,700",85000.0,85300.0,72600.0,214229.0,326030,SK바이오팜
12,2024.08.02,86100.0,"하락 1,000",86100.0,87900.0,85100.0,119535.0,326030,SK바이오팜
13,2024.08.01,87100.0,"상승 1,400",85800.0,88400.0,85300.0,101239.0,326030,SK바이오팜


## 컬럼 순서 변경하기

In [25]:
# DataFrame에서 column 들의 이름을 순서를 조정하여 column순서를 변경할 수 있습니다.
df.columns

Index(['날짜', '종가', '전일비', '시가', '고가', '저가', '거래량', '종목코드', '종목명'], dtype='object')

In [26]:
cols = ['종목코드', '종목명', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
df = df[cols]
df.shape

(1017, 9)

In [27]:
df

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
1,326030,SK바이오팜,2024.08.14,97000.0,보합0,97400.0,98900.0,96300.0,93944.0
2,326030,SK바이오팜,2024.08.13,97000.0,"하락 1,800",98000.0,99800.0,95300.0,393782.0
3,326030,SK바이오팜,2024.08.12,98800.0,"상승 6,500",94200.0,99200.0,93000.0,707165.0
4,326030,SK바이오팜,2024.08.09,92300.0,"상승 2,500",91000.0,93400.0,89300.0,342745.0
5,326030,SK바이오팜,2024.08.08,89800.0,"상승 4,300",84500.0,92900.0,82800.0,698388.0
9,326030,SK바이오팜,2024.08.07,85500.0,"상승 6,400",77600.0,85700.0,77500.0,180833.0
10,326030,SK바이오팜,2024.08.06,79100.0,"상승 1,700",79300.0,82200.0,78200.0,163093.0
11,326030,SK바이오팜,2024.08.05,77400.0,"하락 8,700",85000.0,85300.0,72600.0,214229.0
12,326030,SK바이오팜,2024.08.02,86100.0,"하락 1,000",86100.0,87900.0,85100.0,119535.0
13,326030,SK바이오팜,2024.08.01,87100.0,"상승 1,400",85800.0,88400.0,85300.0,101239.0


## 중복데이터 제거하기
* drop_duplicates 를 통해 중복된 데이터가 있다면 제거합니다.

In [28]:
df[df.duplicated()==True]

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량


In [29]:
# drop_duplicates : row들 끼리 data를 비교하여 같은 값이 있으면 row중 하나를 삭제
# df.shape를 전/후로 출력하여 삭제된 row가 있는지 확인
df = df.drop_duplicates()
df.shape

(1017, 9)

## 기술통계값 구하기

In [30]:
# describe는 기본적으로 수치데이터에 대한 기술통계값을 구하게 됩니다.
df.describe()

,종가,시가,고가,저가,거래량
count,1017.000000,1017.000000,1017.000000,1017.000000,1.017000e+03
mean,100211.602753,100335.988201,102014.945919,98614.650934,2.668842e+05
std,33731.240968,34000.450478,34737.785096,33075.709378,6.355843e+05
min,51800.000000,52500.000000,53600.000000,50900.000000,3.752600e+04
25%,77500.000000,77600.000000,78800.000000,76500.000000,9.725700e+04
50%,88700.000000,88700.000000,89900.000000,87600.000000,1.481750e+05
75%,112500.000000,113000.000000,114000.000000,111000.000000,2.409190e+05
max,217000.000000,219000.000000,269500.000000,210000.000000,1.242668e+07


* 참고 : E notation
[과학적 기수법 - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%EA%B3%BC%ED%95%99%EC%A0%81_%EA%B8%B0%EC%88%98%EB%B2%95)

과학적 기수법, 과학적 표기법(scientific notation, scientific form, standard index form, standard form)은 너무 크거나 너무 작은 숫자들을 십진법으로 편하게 작성하여 표현하는 방법이다. 과학자, 수학자, 공학자들이 공통적으로 사용하는데, 부분적인 이유는 특정한 산술을 단순화시켜 주기 때문이다. 과학 계산기에서는 "SCI" 디스플레이 모드라는 이름으로 알려져 있다.

In [31]:
# 거래량의 과학적 기수법 읽기
1.000000e+02

100.0

In [32]:
4.701523e+05

470152.3

In [33]:
4.701523 * (10 ** 5)

470152.3

In [34]:
# 과학적 표기법 대신 소수점 5자리까지 나타낸다.
pd.options.display.float_format = '{:.5f}'.format

# 다시 원래대로 옵션을 변경하고 싶을 때는 아래 명령어를 사용하면 됩니다.
#pd.reset_option('display.float_format')

df.describe()

,종가,시가,고가,저가,거래량
count,1017.00000,1017.00000,1017.00000,1017.00000,1017.00000
mean,100211.60275,100335.98820,102014.94592,98614.65093,266884.23304
std,33731.24097,34000.45048,34737.78510,33075.70938,635584.34753
min,51800.00000,52500.00000,53600.00000,50900.00000,37526.00000
25%,77500.00000,77600.00000,78800.00000,76500.00000,97257.00000
50%,88700.00000,88700.00000,89900.00000,87600.00000,148175.00000
75%,112500.00000,113000.00000,114000.00000,111000.00000,240919.00000
max,217000.00000,219000.00000,269500.00000,210000.00000,12426682.00000


## 최근 날짜 구해서 파일명 만들기

In [35]:
df.iloc[0]

종목코드        326030
종목명         SK바이오팜
날짜      2024.08.14
종가     97000.00000
전일비            보합0
시가     97400.00000
고가     98900.00000
저가     96300.00000
거래량    93944.00000
Name: 1, dtype: object

In [36]:
# 날짜 column의 첫 row값 확인
date = df.iloc[0]["날짜"]
date

'2024.08.14'

In [37]:
# 종목명, 종목코드, 날짜를 이름으로 하는 csv 파일명 만들기
file_name = f"data/{item_name}_{item_code}_{date}.csv"
file_name

'data/SK바이오팜_326030_2024.08.14.csv'

## 파일로 저장하기

In [38]:
# 파일로 저장하기 
# index=False 로 데이터프레임의 기본 index 는 저장하지 않도록 합니다.
df.to_csv(file_name, index=False)

In [39]:
# 제대로 저장되었는지 파일을 읽어서 확인합니다
pd.read_csv(file_name)

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
0,326030,SK바이오팜,2024.08.14,97000.00000,보합0,97400.00000,98900.00000,96300.00000,93944.00000
1,326030,SK바이오팜,2024.08.13,97000.00000,"하락 1,800",98000.00000,99800.00000,95300.00000,393782.00000
2,326030,SK바이오팜,2024.08.12,98800.00000,"상승 6,500",94200.00000,99200.00000,93000.00000,707165.00000
3,326030,SK바이오팜,2024.08.09,92300.00000,"상승 2,500",91000.00000,93400.00000,89300.00000,342745.00000
4,326030,SK바이오팜,2024.08.08,89800.00000,"상승 4,300",84500.00000,92900.00000,82800.00000,698388.00000
5,326030,SK바이오팜,2024.08.07,85500.00000,"상승 6,400",77600.00000,85700.00000,77500.00000,180833.00000
6,326030,SK바이오팜,2024.08.06,79100.00000,"상승 1,700",79300.00000,82200.00000,78200.00000,163093.00000
7,326030,SK바이오팜,2024.08.05,77400.00000,"하락 8,700",85000.00000,85300.00000,72600.00000,214229.00000
8,326030,SK바이오팜,2024.08.02,86100.00000,"하락 1,000",86100.00000,87900.00000,85100.00000,119535.00000
9,326030,SK바이오팜,2024.08.01,87100.00000,"상승 1,400",85800.00000,88400.00000,85300.00000,101239.00000


## 전체 과정을 하나의 함수로 만들기

In [40]:
def get_item_list(item_code, item_name):
    """
    일별 시세를 수집하는 함수
    """
    # web page 시작번호
    page_no = 1
    # 데이터를 저장할 빈 변수 선언
    item_list = []

    while True:
        temp = get_day_list(item_code, page_no)
        item_list.append(temp)

        page_no = page_no + 1
        time.sleep(0.1)

        #if temp.shape[0] < 10:
        if page_no > 10:
            break
            
    df = pd.concat(item_list)
    df["종목코드"] = item_code
    df["종목명"] = item_name
    cols = ['종목코드', '종목명', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
    df = df[cols]

    return df

In [41]:
item_code = "352820"
item_name = "하이브"

# item_code = "326030"
# item_name = "SK바이오팜"

get_item_list(item_code, item_name)

C:\Users\salix\AppData\Local\Temp\ipykernel_15824\4100071483.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))
C:\Users\salix\AppData\Local\Temp\ipykernel_15824\4100071483.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))
C:\Users\salix\AppData\Local\Temp\ipykernel_15824\4100071483.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))
C:\Users\salix\AppData\Local\Temp\ipykernel_15824\4100071483.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
1,352820,하이브,2024.08.14,167500.00000,"하락 3,400",171000.00000,171300.00000,167400.00000,85241.00000
2,352820,하이브,2024.08.13,170900.00000,"하락 1,200",172100.00000,173700.00000,170600.00000,119427.00000
3,352820,하이브,2024.08.12,172100.00000,하락 100,172300.00000,173500.00000,170000.00000,189214.00000
4,352820,하이브,2024.08.09,172200.00000,"하락 11,600",183800.00000,183800.00000,171600.00000,371507.00000
5,352820,하이브,2024.08.08,183800.00000,"상승 2,300",178800.00000,184900.00000,174100.00000,336637.00000
9,352820,하이브,2024.08.07,181500.00000,"상승 10,100",169100.00000,189100.00000,168500.00000,581910.00000
10,352820,하이브,2024.08.06,171400.00000,"상승 1,300",170300.00000,176500.00000,167000.00000,288864.00000
11,352820,하이브,2024.08.05,170100.00000,"하락 10,700",179000.00000,180400.00000,160000.00000,314374.00000
12,352820,하이브,2024.08.02,180800.00000,"상승 5,500",174500.00000,184000.00000,174500.00000,359084.00000
13,352820,하이브,2024.08.01,175300.00000,하락 700,174300.00000,176800.00000,174300.00000,70054.00000


## 내가 만든 함수의 기능과 소스코드가 궁금하다면?

In [42]:
# 도움말 보기
get_item_list?

In [43]:
# 소스코드 보기
get_item_list??

In [44]:
# 소스코드 보기
get_day_list??

#### 실습(1) 읽어온 데이터프레임을 파일로 저장하는 코드 추가

In [45]:
def df_to_file(item_code, item_name):
    """
    일별 시세를 수집하는 함수
    """
    df = get_item_list(item_code, item_name)
    file_name = f"data/{item_name}_{item_code}_{date}.csv"
    df.to_csv(file_name, index=False)

#### 실습(2)  2개이상 여러 종목 데이터를 불러오는 코드 작성

In [46]:
# 불러올 종가의 목록 입니다.
stock_dict = {
    '삼성전자': '005930',
    'SK하이닉스': '000660',
    '현대차': '005380',
    '셀트리온': '068270',
    'LG전자': '066570',
    'POSCO': '005490',
    '삼성물산': '028260',
    'NAVER': '035420'}
stock_dict

{'삼성전자': '005930',
 'SK하이닉스': '000660',
 '현대차': '005380',
 '셀트리온': '068270',
 'LG전자': '066570',
 'POSCO': '005490',
 '삼성물산': '028260',
 'NAVER': '035420'}

In [47]:
stock_dict.items()

dict_items([('삼성전자', '005930'), ('SK하이닉스', '000660'), ('현대차', '005380'), ('셀트리온', '068270'), ('LG전자', '066570'), ('POSCO', '005490'), ('삼성물산', '028260'), ('NAVER', '035420')])

In [48]:
list(stock_dict.items())[0][1]

'005930'

In [49]:
# 반복문을 사용하여 여러 종목의 데이터를 수집합니다.
for item in stock_dict.items():
    item_code = str(list(item)[1])
    item_name = str(list(item)[0])    
    #print(item_code)
    #print(item_name)
    df_to_file(item_code, item_name)
    print(item_name+" is completed - item_code is : "+item_code)

C:\Users\salix\AppData\Local\Temp\ipykernel_15824\4100071483.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))
C:\Users\salix\AppData\Local\Temp\ipykernel_15824\4100071483.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))
C:\Users\salix\AppData\Local\Temp\ipykernel_15824\4100071483.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))
C:\Users\salix\AppData\Local\Temp\ipykernel_15824\4100071483.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

삼성전자 is completed - item_code is : 005930


C:\Users\salix\AppData\Local\Temp\ipykernel_15824\4100071483.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))
C:\Users\salix\AppData\Local\Temp\ipykernel_15824\4100071483.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))
C:\Users\salix\AppData\Local\Temp\ipykernel_15824\4100071483.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))
C:\Users\salix\AppData\Local\Temp\ipykernel_15824\4100071483.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

SK하이닉스 is completed - item_code is : 000660


C:\Users\salix\AppData\Local\Temp\ipykernel_15824\4100071483.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))
C:\Users\salix\AppData\Local\Temp\ipykernel_15824\4100071483.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))
C:\Users\salix\AppData\Local\Temp\ipykernel_15824\4100071483.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))
C:\Users\salix\AppData\Local\Temp\ipykernel_15824\4100071483.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

...

POSCO is completed - item_code is : 005490


C:\Users\salix\AppData\Local\Temp\ipykernel_15824\4100071483.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))
C:\Users\salix\AppData\Local\Temp\ipykernel_15824\4100071483.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))
C:\Users\salix\AppData\Local\Temp\ipykernel_15824\4100071483.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))
C:\Users\salix\AppData\Local\Temp\ipykernel_15824\4100071483.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

삼성물산 is completed - item_code is : 028260


C:\Users\salix\AppData\Local\Temp\ipykernel_15824\4100071483.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))
C:\Users\salix\AppData\Local\Temp\ipykernel_15824\4100071483.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))
C:\Users\salix\AppData\Local\Temp\ipykernel_15824\4100071483.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))
C:\Users\salix\AppData\Local\Temp\ipykernel_15824\4100071483.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

NAVER is completed - item_code is : 035420


In [50]:
## 파일 검증
file_name = f"data/{item_name}_{item_code}_{date}.csv"
print(file_name)
pd.read_csv(file_name)

data/NAVER_035420_2024.08.14.csv


,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
0,35420,NAVER,2024.08.14,156300.00000,하락 100,157300.00000,158000.00000,155700.00000,290767.00000
1,35420,NAVER,2024.08.13,156400.00000,"하락 3,900",159800.00000,160300.00000,156100.00000,1035007.00000
2,35420,NAVER,2024.08.12,160300.00000,"하락 3,400",164600.00000,164700.00000,159400.00000,811411.00000
3,35420,NAVER,2024.08.09,163700.00000,"상승 1,000",170000.00000,170000.00000,163100.00000,1236721.00000
4,35420,NAVER,2024.08.08,162700.00000,보합0,160100.00000,163500.00000,158900.00000,724039.00000
5,35420,NAVER,2024.08.07,162700.00000,"상승 4,300",156100.00000,163500.00000,156000.00000,778054.00000
6,35420,NAVER,2024.08.06,158400.00000,"상승 2,300",160000.00000,163300.00000,156600.00000,1115733.00000
7,35420,NAVER,2024.08.05,156100.00000,"하락 15,300",169000.00000,169600.00000,151100.00000,1447269.00000
8,35420,NAVER,2024.08.02,171400.00000,"하락 3,600",172000.00000,172500.00000,170500.00000,674484.00000
9,35420,NAVER,2024.08.01,175000.00000,"상승 1,000",176500.00000,177000.00000,174500.00000,481980.00000
